In [1]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, KBinsDiscretizer
from aif360.sklearn.datasets import fetch_adult
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

In [2]:
X = fetch_adult()
X

/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


WeightedDataset(X=                   age     workclass     education  education-num  \
race      sex                                                       
Non-white Male    25.0       Private          11th            7.0   
White     Male    38.0       Private       HS-grad            9.0   
          Male    28.0     Local-gov    Assoc-acdm           12.0   
Non-white Male    44.0       Private  Some-college           10.0   
White     Male    34.0       Private          10th            6.0   
...                ...           ...           ...            ...   
          Female  27.0       Private    Assoc-acdm           12.0   
          Male    40.0       Private       HS-grad            9.0   
          Female  58.0       Private       HS-grad            9.0   
          Male    22.0       Private       HS-grad            9.0   
          Female  52.0  Self-emp-inc       HS-grad            9.0   

                      marital-status         occupation   relationship   race  \
rac

In [3]:
import pandas as pd
from sklearn import preprocessing

def load_adult(drop_protected=True):
    data_df = pd.read_csv(f"adult.csv")
    
    # Drop the target column
    TARGET_COLUMNS = data_df.columns[-1]
    data = data_df.drop(columns=[TARGET_COLUMNS])

    # Drop the 'sex' (gender) column if the protected attribute needs to be hidden
    if drop_protected:
        if 'sex' in data.columns:
            data = data.drop(columns=['sex'])  # Drop the gender column
    
    # Preprocess the dataset (assuming preprocess_dataset is defined elsewhere)
    data, numeric_columns, categorical_columns = preprocess_dataset(data, continuous_features=[])

    data_df_copy = data.copy()
    
    # Scale the dataset
    min_max_scaler = preprocessing.MinMaxScaler()
    data_scaled = min_max_scaler.fit_transform(data)
    data = pd.DataFrame(data_scaled, columns=data.columns)

    FEATURE_COLUMNS = data.columns

    # Add the target column back
    data[TARGET_COLUMNS] = data_df[TARGET_COLUMNS]

    return data, FEATURE_COLUMNS, TARGET_COLUMNS, numeric_columns, categorical_columns

In [4]:
def preprocess_dataset(df, continuous_features=[]):
    label_encoder = LabelEncoder()
    onehot_encoder = OneHotEncoder()

    numeric_columns = []
    categorical_columns = []

    # Iterate over each column in the DataFrame
    for col in df.columns:
        # Check if the column is categorical
        if df[col].dtype == 'object' or df[col].dtype == 'category' and col not in continuous_features:
            categorical_columns.append(col)
            # If the column has only two unique values, treat it as binary categorical
            if len(df[col].unique()) == 2:
                # Label encode binary categorical features
                df[col] = label_encoder.fit_transform(df[col])
            else:
                # One-hot encode regular categorical features
                encoded_values = onehot_encoder.fit_transform(df[[col]])
                # Create new column names for the one-hot encoded features
                new_cols = [col + '_' + str(i) for i in range(encoded_values.shape[1])]
                # Convert the encoded values to a DataFrame and assign column names
                encoded_df = pd.DataFrame(encoded_values.toarray(), columns=new_cols)
                # Concatenate the encoded DataFrame with the original DataFrame
                df = pd.concat([df, encoded_df], axis=1)
                # Drop the original categorical column from the DataFrame
                df.drop(col, axis=1, inplace=True)
        # If the column is numerical but in string format and not in continuous_features, convert it to numerical type
        elif df[col].dtype == 'object' or df[col].dtype == 'category' and df[
            col].str.isnumeric().all() and col not in continuous_features:
            df[col] = df[col].astype(int)  # Convert to integer type
            categorical_columns.append(col)
        # If the column is a continuous feature, discretize it into bins
        elif col in continuous_features:
            numeric_columns.append(col)
            # Calculate the number of bins
            num_unique_values = len(df[col].unique())
            value_range = df[col].max() - df[col].min()
            num_bins = calculate_num_bins(num_unique_values, value_range)

            # Discretize into bins
            bin_discretizer = KBinsDiscretizer(n_bins=num_bins, encode='ordinal', strategy='uniform')
            bins = bin_discretizer.fit_transform(df[[col]])
            # Replace the original continuous feature with the binned values
            df[col] = bins.astype(int)
        else:
            # Here are numerical columns. If the column has only 2 unique values, dont add it to numeric_columns
            if len(df[col].unique()) > 2:
                numeric_columns.append(col)
    return df, numeric_columns, categorical_columns

In [5]:
data = load_adult(drop_protected=True)
data

(            age  workclass  education  education-num  marital-status  \
 0      0.301370      0.875   0.600000       0.800000        0.666667   
 1      0.452055      0.750   0.600000       0.800000        0.333333   
 2      0.287671      0.500   0.733333       0.533333        0.000000   
 3      0.493151      0.500   0.066667       0.400000        0.333333   
 4      0.150685      0.500   0.600000       0.800000        0.333333   
 ...         ...        ...        ...            ...             ...   
 48837  0.301370      0.500   0.600000       0.800000        0.000000   
 48838  0.643836      0.000   0.733333       0.533333        1.000000   
 48839  0.287671      0.500   0.600000       0.800000        0.333333   
 48840  0.369863      0.500   0.600000       0.800000        0.000000   
 48841  0.246575      0.625   0.600000       0.800000        0.333333   
 
        occupation  relationship  race  capital-gain  capital-loss  \
 0        0.071429           0.2  1.00      0.021740

In [6]:
data, FEATURE_COLUMNS, TARGET_COLUMNS, numeric_columns, categorical_columns = load_adult(drop_protected=True)

print(data.columns)

Index(['age', 'workclass', 'education', 'education-num', 'marital-status',
       'occupation', 'relationship', 'race', 'capital-gain', 'capital-loss',
       'hours-per-week', 'target'],
      dtype='object')


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def calculate_metrics_and_split_data(data, features, target, classifier_type='logistic_regression',
                                     metrics=['accuracy'], attribute=None):
    """
    Train a classifier, predict on test data, calculate specified metrics from the confusion matrix,
    and optionally split the test data based on an attribute.

    :param data: DataFrame containing the dataset
    :param features: List of feature column names
    :param target: Name of the target column
    :param classifier_type: Type of classifier ('logistic_regression', 'svm', 'random_forest', 'naive_bayes')
    :param metrics: List of metrics to calculate ('accuracy', 'precision', 'recall', 'f1', 'tp', 'tn', 'fp', 'fn')
    :param attribute: Optional attribute to split the test data on, which should be binary (0 or 1)
    :return: Dictionary of requested metrics and optionally two DataFrames, one for each value of the attribute,
             and the trained model instance
    """
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data[FEATURE_COLUMNS], data[TARGET_COLUMNS], test_size=0.2, random_state=42)

    # Debugging: Print the shape of the train and test sets
    print(f"Training data shape: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Testing data shape: X_test: {X_test.shape}, y_test: {y_test.shape}")

    # Initialize classifier
    classifiers = {
        'logistic_regression': LogisticRegression(max_iter=500, solver='saga'),
        'svm': SVC(),
        'random_forest': RandomForestClassifier(),
        'naive_bayes': GaussianNB()
    }
    classifier = classifiers[classifier_type]

    # Fit the classifier and predict
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    tp = cm[1, 1]  # True Positives
    tn = cm[0, 0]  # True Negatives
    fp = cm[0, 1]  # False Positives
    fn = cm[1, 0]  # False Negatives

    # Debugging: Print the confusion matrix
    print("Confusion Matrix:")
    print(cm)

    # Calculate metrics
    metrics_dict = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred, average='binary'),
        'recall': recall_score(y_test, y_pred, average='binary'),
        'f1': f1_score(y_test, y_pred, average='binary'),
        'tp': tp,
        'tn': tn,
        'fp': fp,
        'fn': fn
    }

    # Print accuracy
    print(f"Accuracy: {metrics_dict['accuracy']}")

    # Validate metrics
    results = {}
    for metric in metrics:
        if metric not in metrics_dict:
            raise ValueError(
                f"Unsupported metric '{metric}'. Choose from 'accuracy', 'precision', 'recall', 'f1', 'tp', 'tn', 'fp', 'fn'.")
        results[metric] = metrics_dict[metric]

#     if attribute is not None:
#         if attribute != target:
#             raise ValueError(f"The attribute for splitting should be the same as the target.")

        # Add target to test features
        X_test_df = X_test.copy()
        X_test_df[target] = y_test.values

#         # Split the test data into two groups
#         group_0 = X_test_df[X_test_df[attribute] == 0]
#         group_1 = X_test_df[X_test_df[attribute] == 1]

#         return results, group_0, group_1, classifier

    # Add the target column back to the test set for completeness
    X_test_df = X_test.copy()
    X_test_df[target] = y_test.values
    X_test_df['pred'] = y_pred

    # Debugging: Print the test data with all columns
    print("Test data with all columns:")
    print(X_test_df.head())

    return results, X_test_df, classifier

In [8]:
def generate_tn_fn_dataframes_without_split(test_df, target):
    # Confusion matrix for the entire dataset
    y_true = test_df[target]
    y_pred = test_df['pred']
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print(f"Confusion Matrix: TN={tn}, FP={fp}, FN={fn}, TP={tp}")

    # Create DataFrames for TN and FN
    tn_df = test_df[(test_df[target] == 0) & (test_df['pred'] == 0)].drop(columns=[target, 'pred'])
    fn_df = test_df[(test_df[target] == 1) & (test_df['pred'] == 0)].drop(columns=[target, 'pred'])

    print(f"TN size: {tn_df.shape[0]}")
    print(f"FN size: {fn_df.shape[0]}")
    return tn_df, fn_df

In [12]:
data, features, target, numeric_columns, categorical_columns = load_adult(drop_protected=True)

target = 'target'
attribute = None
classifier_type = 'logistic_regression'

# Calculate metrics and get the test DataFrame with predictions
results, test_df, trained_model = calculate_metrics_and_split_data(data, features, target, classifier_type, attribute=attribute)

# Generate DataFrames for true negatives and false negatives
tn_df, fn_df = generate_tn_fn_dataframes_without_split(test_df, target)
print("Results:")
for metric, value in results.items():
    print(f"{metric}: {value}")
print("True Negatives (first 5 rows):")
print(tn_df.head())
print("False Negatives (first 5 rows):")
print(fn_df.head())

Training data shape: X_train: (39073, 11), y_train: (39073,)
Testing data shape: X_test: (9769, 11), y_test: (9769,)
Confusion Matrix:
[[1002 1353]
 [ 396 7018]]
Accuracy: 0.8209642747466476
Test data with all columns:
            age  workclass  education  education-num  marital-status  \
7762   0.013699       0.50   0.733333       0.533333        0.666667   
23881  0.000000       0.50   0.133333       0.466667        0.666667   
30507  0.109589       0.25   0.733333       0.533333        0.666667   
28911  0.041096       0.50   1.000000       0.600000        0.666667   
19484  0.410959       0.50   0.733333       0.533333        0.666667   

       occupation  relationship  race  capital-gain  capital-loss  \
7762     0.571429           0.2   1.0           0.0           0.0   
23881    0.857143           0.6   1.0           0.0           0.0   
30507    0.428571           0.4   0.5           0.0           0.0   
28911    0.857143           0.6   1.0           0.0           0.0   
194

In [13]:
import dice_ml

d = dice_ml.Data(dataframe=data,
                 continuous_features=["education-num", "hours-per-week"],
                 outcome_name='target')

In [14]:
m = dice_ml.Model(model=trained_model, backend="sklearn")

In [15]:
explainer = dice_ml.Dice(d, m, method="random")

In [23]:
input_datapoint = tn_df[0:1]
tn_df_cf = explainer.generate_counterfactuals(input_datapoint, 
                                  total_CFs=10, 
                                  desired_class="opposite",
)
# Visualize it
tn_df_cf.visualize_as_dataframe(show_only_changes=True)

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 10.31it/s]

Query instance (original outcome : 0)


,age,workclass,education,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,target
0,0.39726,0.5,0.6,0.8,0.333333,0.857143,0.0,1.0,0.0,0.453857,0.397959,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,target
0,-,-,-,0.11341486,-,-,-,-,-,-,-,1.0
1,-,-,-,0.108506,-,-,-,-,-,-,0.61964487,1.0
2,-,0.75,-,0.27426485,-,-,-,-,-,-,-,1.0
3,-,0.625,-,0.03368003,-,-,-,-,-,-,-,1.0
4,-,-,-,0.02869702,-,-,-,-,-,-,0.32556348,1.0
5,-,-,-,0.01151581,-,-,-,-,-,-,0.27551615,1.0
6,-,-,-,0.56807239,0.6666666666666666,-,-,-,-,-,-,1.0
7,-,-,-,0.16467514,-,-,-,-,-,-,-,1.0
8,-,1.0,-,0.14559422,-,-,-,-,-,-,-,1.0
9,-,-,-,0.37729244,-,-,-,-,-,-,-,1.0


In [22]:
input_datapoint = fn_df[0:1]
fn_df_cf = explainer.generate_counterfactuals(input_datapoint, 
                                  total_CFs=10, 
                                  desired_class="opposite",
)
# Visualize it
fn_df_cf.visualize_as_dataframe(show_only_changes=True)

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.65s/it]

Query instance (original outcome : 0)


,age,workclass,education,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,target
0,0.356164,0.5,0.466667,0.733333,0.0,0.714286,0.2,1.0,0.0465,0.0,0.397959,0



Diverse Counterfactual set (new outcome: 1)


,age,workclass,education,education-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,target
0,-,-,-,0.5047061,-,-,-,-,-,-,-,1.0
1,-,-,-,0.20505809,-,-,-,-,0.034640346403464035,-,-,1.0
2,-,-,-,0.12634076,-,-,-,-,-,-,-,1.0
3,0.3424657534246575,-,-,0.22942299,-,-,-,-,-,-,-,1.0
4,-,-,-,0.08876807,-,-,-,-,-,-,-,1.0
5,-,-,-,0.6695056900005025,0.6666666666666666,-,-,-,-,-,-,1.0
6,-,0.625,-,-,0.8333333333333333,-,-,-,-,-,-,1.0
7,0.1643835616438356,-,-,-,-,-,-,-,0.034560345603456034,-,-,1.0
8,-,-,-,0.41035762,-,-,-,-,-,-,-,1.0
9,-,-,-,-,0.5,0.5,-,-,-,-,-,1.0


In [24]:
def burden_distance(counterfactuals_df, input_datapoint):
    if isinstance(input_datapoint, pd.DataFrame):
        input_datapoint = input_datapoint.iloc[0]
    
    # Calculate the difference for each counterfactual row
    differences = counterfactuals_df.subtract(input_datapoint, axis=1)
    
    # If the difference is not zero (meaning the value has changed), take the absolute value; otherwise, ignore.
    distances = differences.where(differences != 0, 0).abs()
    
    # Sum the distances for each counterfactual row
    burden_distances = distances.sum(axis=1)
    
    mean_burden_distance = burden_distances.mean()
    
    return burden_distances, mean_burden_distance

In [25]:
distances = burden_distance(tn_df_cf.cf_examples_list[0].final_cfs_df, input_datapoint)
print(distances)

(0    0.686585
1    0.913180
2    0.775735
3    0.891320
4    0.843699
5    0.910927
6    0.565261
7    0.635325
8    1.154406
9    0.422708
dtype: float64, 0.7799144997006804)


In [26]:
distances = burden_distance(fn_df_cf.cf_examples_list[0].final_cfs_df, input_datapoint)
print(distances)

(0    1.646271
1    1.934059
2    2.024636
3    1.935253
4    2.062209
5    1.481571
6    1.709310
7    1.597484
8    1.740619
9    1.465263
dtype: float64, 1.759667475084526)


In [28]:
# Get the original prediction
original_prediction = trained_model.predict(input_datapoint)[0]

# Extract the counterfactual data and predictions (DiCE-specific method)
cf_dataframe = tn_df_cf.cf_examples_list[0].final_cfs_df
cf_predictions = cf_dataframe['predicted_outcome'].values

# Calculate the prediction difference
cf_dataframe['prediction_difference'] = cf_predictions - original_prediction

# Print or plot the DataFrame for debugging
print(cf_dataframe[['prediction_difference']])

KeyError: 'predicted_outcome'